# Prepare excel read with pandas

In [1]:
import pandas as pd
# data = pd.read_excel(r'monthly_market_data - Copy.xlsx')
data = pd.read_excel(r'monthly_market_data - Copy (3).xlsx')

# print(data)


# Read Excel to memnory and create classes

In [2]:
import operator

class CompanyInfo:
    def __init__(self, tickerKey, name, month,numberOfOutstandingShares, marketcap,b2m, adjClose,inv,op,sourceSubsectorCode
):
        self.tickerKey = tickerKey
        self.name = name
        self.month = month
        self.rollingAverage = None
        self.numberOfOutstandingShares = numberOfOutstandingShares if not pd.isnull(numberOfOutstandingShares) else None
        self.marketcap = marketcap if not pd.isnull(marketcap) else None
        self.b2m = b2m if not pd.isnull(b2m) else None
        self.closeAdjust = adjClose
        self.momentum = None
        self.size = None
        self.RMWSize = None
        self.CMASize = None
        self.BperMSize = None
        self.currentYield = None
        self.inv = inv if not pd.isnull(inv) else None
        self.op = op if not pd.isnull(op) else None
        self.sourceSubsectorCode = sourceSubsectorCode


    def __repr__(self):
         return self.__str__()

    def __str__(self):
        return "tickerKey:"+str(self.tickerKey) + \
            "\t month:" + str(self.month) + \
            "\t closeAdjust:" + str(self.closeAdjust) + \
            "\t currentYield:" + str(self.currentYield) + \
            "\t numberOfOutstandingShares:" + str(self.numberOfOutstandingShares) + \
            "\t marketcap:" + str(self.marketcap) + \
            "\t b2m:" + str(self.b2m) + \
            "\t rollingAverage:" + str(self.rollingAverage) + \
            "\t momentum:" + str(self.momentum) + \
            "\t size:" + str(self.size)+"\n"

    def convertToList(self):
      return [getattr(self,atr) for atr in self.__dict__]


companyInfoDict  = {}
def addCompanyInfoToList(companyInfo:CompanyInfo):
    if companyInfo.tickerKey not in companyInfoDict:
      companyInfoDict[companyInfo.tickerKey]={}
    if companyInfo.month not in companyInfoDict[companyInfo.tickerKey]:
      companyInfoDict[companyInfo.tickerKey][companyInfo.month]=companyInfo
    else:
      raise Exception("Company info exists." + str(companyInfo))

def findCompanyInfo(tickerKey, month):
    if tickerKey in companyInfoDict and month in companyInfoDict[tickerKey]:
      return companyInfoDict[tickerKey][month]
    return None

def displayCompanyInfoHeader():
  for tickerKey in companyInfoDict.keys():
      for month in companyInfoDict[tickerKey].keys():
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        attrs = []
        for attribute in vars(company):
          attrs.append(attribute)
        print(attrs)
        return

def displayCompanyInfo(tickerKeys:list,months:list):
    displayCompanyInfoHeader()
    for tickerKey in tickerKeys:
      sortedMonths = sorted(months)
      for month in sortedMonths:
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        print(company.convertToList())

allMonths = []


for i, companyInfoPandas in data.iterrows():
    ci = CompanyInfo(tickerKey=companyInfoPandas["TickerKey"],
                                       name=companyInfoPandas["TickerNamePooyaFA"],
                                       month=companyInfoPandas["DayKeyFA"]// 100,
                                       numberOfOutstandingShares=companyInfoPandas["NumberOfOutstandingShares"],
                                       marketcap=companyInfoPandas["marketcap"],
                                       b2m=companyInfoPandas["B2M"],
                                       adjClose=companyInfoPandas["AdjClose"],
                                       inv=companyInfoPandas["INV"],
                                       op=companyInfoPandas["OP"],
                                       sourceSubsectorCode=companyInfoPandas["SourceSubsectorCode"],
                                       )
    addCompanyInfoToList(ci)
    allMonths.append(ci.month)

allMonths = list(set(allMonths))
allMonths.sort()


In [3]:
# companies to excel
def exportCompanyListToExcel(excelFileName):
  rows = []
  for company in companyInfoDict.keys():
    for month in companyInfoDict[company].keys():
      rows.append(companyInfoDict[company][month].convertToList())

  company_ = None
  for tickerKey in companyInfoDict.keys():
      for month in sorted(companyInfoDict[company].keys()):
        company:CompanyInfo = findCompanyInfo(tickerKey,month)
        if company is None:
          continue
        company_ = company
        break
      if company_ is not None:
        break
  dfOut = pd.DataFrame(rows,columns=company_.__dict__)
  dfOut.to_excel(excelFileName+'.xlsx', sheet_name='TMPExport')

# Dict access sample and test

In [4]:
print(list(companyInfoDict.keys())[:10])
print(list(companyInfoDict[1].keys())[:10])
print(companyInfoDict[1][139712])


[1, 9, 10, 219, 223, 231, 234, 239, 243, 247]
[139712, 139801, 139802, 139803, 139804, 139805, 139806, 139807, 139808, 139809]
tickerKey:1	 month:139712	 closeAdjust:802.5232992867204	 currentYield:None	 numberOfOutstandingShares:3000000000	 marketcap:4959000000000	 b2m:1.2906721802	 rollingAverage:None	 momentum:None	 size:None



# Extract company names

In [5]:
companyTickerSet = list(companyInfoDict.keys())
print(companyTickerSet)


[1, 9, 10, 219, 223, 231, 234, 239, 243, 247, 251, 255, 259, 263, 267, 271, 275, 279, 283, 287]


# Extract company months

In [6]:
print(allMonths)
print("Total:",len(allMonths))

[138001, 138002, 138003, 138004, 138005, 138006, 138007, 138008, 138009, 138010, 138011, 138012, 138101, 138102, 138103, 138104, 138105, 138106, 138107, 138108, 138109, 138110, 138111, 138112, 138201, 138202, 138203, 138204, 138205, 138206, 138207, 138208, 138209, 138210, 138211, 138212, 138301, 138302, 138303, 138304, 138305, 138306, 138307, 138308, 138309, 138310, 138311, 138312, 138401, 138402, 138403, 138404, 138405, 138406, 138407, 138408, 138409, 138410, 138411, 138412, 138501, 138502, 138503, 138504, 138505, 138506, 138507, 138508, 138509, 138510, 138511, 138512, 138601, 138602, 138603, 138604, 138605, 138606, 138607, 138608, 138609, 138610, 138611, 138612, 138701, 138702, 138703, 138704, 138705, 138706, 138707, 138708, 138709, 138710, 138711, 138712, 138801, 138802, 138803, 138804, 138805, 138806, 138807, 138808, 138809, 138810, 138811, 138812, 138901, 138902, 138903, 138904, 138905, 138906, 138907, 138908, 138909, 138910, 138911, 138912, 139001, 139002, 139003, 139004, 139005,

In [7]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'size', 'RMWSize', 'CMASize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000, 460800000000, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, 5.9385362299, 6.9385362299, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000, 481500000000, 5.1603845371, 10.744208210092395, None, None, None, None, None, None, 6.1603845371, 7.1603845371, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000, 494910000000, 5.3041036578, 11.043439429401511, None, None, None, None, None, None, 6.3041036578, 7.3041036578, 4424]


# Fill missing close adjust

In [8]:
def calculateCloseAdjust(tickerKey, monthIndex):
  privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])
  if privuseMonthCompanyInfo is None:
    return None
  nextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+1])

  if nextMonthCompanyInfo is not None:
    return ((nextMonthCompanyInfo.closeAdjust / privuseMonthCompanyInfo.closeAdjust) ** (1/2)) * privuseMonthCompanyInfo.closeAdjust

  if monthIndex + 2 >= len(allMonths):
    return None
  secondNextMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex+2])
  if secondNextMonthCompanyInfo is None:
    return None
  return ((secondNextMonthCompanyInfo.closeAdjust / privuseMonthCompanyInfo.closeAdjust) ** (1/3)) * privuseMonthCompanyInfo.closeAdjust
  
  # print(calculatedCloseAdjust)

_missingData = []
for tickerKey in companyTickerSet:
  # companyHistoricalData = [companyInfo for companyInfo in companyInfoList if companyInfo.tickerKey == tickerKey]
  startOfData = False
  for monthIndex in range(1,len(allMonths)-1):
    currentCompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if currentCompanyInfo is not None:
        startOfData = True
        continue
    if currentCompanyInfo is None:
      closeAdjust = calculateCloseAdjust(tickerKey,monthIndex)
      if closeAdjust is None: 
        if startOfData :
          _missingData.append(["closeAdjust for tick",tickerKey," Month",allMonths[monthIndex]," Is None"])
        continue
      privuseMonthCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey, allMonths[monthIndex-1])

      adjustedCompanyInfo = CompanyInfo(tickerKey,privuseMonthCompanyInfo.name,allMonths[monthIndex],None,None,None,None,None,None,privuseMonthCompanyInfo.sourceSubsectorCode)
      adjustedCompanyInfo.closeAdjust = closeAdjust
      # print(tickerKey,allMonths[month],privuseMonthCompanyInfo) # log before adjusted
      if privuseMonthCompanyInfo.numberOfOutstandingShares == None:
        print("None privuse marketcap" ,privuseMonthCompanyInfo)
        continue;
      adjustedCompanyInfo.marketcap = closeAdjust/privuseMonthCompanyInfo.closeAdjust * privuseMonthCompanyInfo.marketcap
      if privuseMonthCompanyInfo.numberOfOutstandingShares == None:
        print("None numberOfOutstandingShares" ,privuseMonthCompanyInfo)
        continue;
      adjustedCompanyInfo.numberOfOutstandingShares = privuseMonthCompanyInfo.numberOfOutstandingShares
      addCompanyInfoToList(adjustedCompanyInfo)
      # print(tickerKey,allMonths[month]) # log adjusted


# print([companyInfo for companyInfo in _missingData if companyInfo[1] == 9])

In [9]:
print("{} of {}".format(len(_missingData),data.size))
print(_missingData[:10])

231 of 52318
[['closeAdjust for tick', 9, ' Month', 138403, ' Is None'], ['closeAdjust for tick', 9, ' Month', 138404, ' Is None'], ['closeAdjust for tick', 9, ' Month', 138405, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139307, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139308, ' Is None'], ['closeAdjust for tick', 223, ' Month', 139309, ' Is None'], ['closeAdjust for tick', 231, ' Month', 138311, ' Is None'], ['closeAdjust for tick', 231, ' Month', 138312, ' Is None'], ['closeAdjust for tick', 231, ' Month', 138401, ' Is None'], ['closeAdjust for tick', 234, ' Month', 138907, ' Is None']]


# Test data

In [10]:

displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'size', 'RMWSize', 'CMASize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000, 460800000000, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, 5.9385362299, 6.9385362299, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000, 481500000000, 5.1603845371, 10.744208210092395, None, None, None, None, None, None, 6.1603845371, 7.1603845371, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000, 494910000000, 5.3041036578, 11.043439429401511, None, None, None, None, None, None, 6.3041036578, 7.3041036578, 4424]


# Yield calculations

In [11]:
for tickerKey in companyTickerSet:
  for monthIndex in range(1,len(allMonths)):
    lastMonthData:CompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex-1])
    if lastMonthData is None:
      continue
    thisMonthData:CompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if thisMonthData is None:
      continue
    thisMonthData.currentYield = thisMonthData.closeAdjust / lastMonthData.closeAdjust - 1

In [12]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'size', 'RMWSize', 'CMASize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000, 460800000000, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, 5.9385362299, 6.9385362299, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000, 481500000000, 5.1603845371, 10.744208210092395, None, None, None, None, None, 0.044921875, 6.1603845371, 7.1603845371, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000, 494910000000, 5.3041036578, 11.043439429401511, None, None, None, None, None, 0.027850467289719738, 6.3041036578, 7.3041036578, 4424]


# Calculate rolling average

In [13]:
def getRollingDataWindow(tickerKey,rollingWindowSize,endMonthIndex):
  rollingDataWindow = []
  for monthIndex in range(endMonthIndex+1 - rollingWindowSize,endMonthIndex+1):
    currentCompanyInfo = findCompanyInfo(tickerKey,allMonths[monthIndex])
    if currentCompanyInfo is None:
        continue
    rollingDataWindow.append(currentCompanyInfo)
  return rollingDataWindow

In [14]:
def average(dataList):
    if len(dataList)==0:
        return None
    data = [info.currentYield for info in dataList]
    if None in data:
      return None
    # print(data)
    return sum(data) / len(dataList)

# print(companyInfoList)
rollingAverageWindowSize = 6     #Can be change
for tickerKey in companyTickerSet:
    # print("tickerKey",tickerKey)
    for endMonthIndex in range(rollingAverageWindowSize-1,len(allMonths)):
      # print("endMonthIndex",endMonthIndex)
      rollingDataWindow = getRollingDataWindow(tickerKey, rollingAverageWindowSize,endMonthIndex)
      if len(rollingDataWindow) != rollingAverageWindowSize: continue
      currentCompanyInfo:CompanyInfo = findCompanyInfo(tickerKey,allMonths[endMonthIndex])
      currentCompanyInfo.rollingAverage = average(rollingDataWindow)

In [15]:
displayCompanyInfo(companyTickerSet[:3],allMonths[:3])

['tickerKey', 'name', 'month', 'rollingAverage', 'numberOfOutstandingShares', 'marketcap', 'b2m', 'closeAdjust', 'momentum', 'size', 'RMWSize', 'CMASize', 'BperMSize', 'currentYield', 'inv', 'op', 'sourceSubsectorCode']
[9, 'پاکسان\u200c', 138001, None, 90000000, 460800000000, 4.9385362299, 10.282307670219264, None, None, None, None, None, None, 5.9385362299, 6.9385362299, 4424]
[9, 'پاکسان\u200c', 138002, None, 90000000, 481500000000, 5.1603845371, 10.744208210092395, None, None, None, None, None, 0.044921875, 6.1603845371, 7.1603845371, 4424]
[9, 'پاکسان\u200c', 138003, None, 90000000, 494910000000, 5.3041036578, 11.043439429401511, None, None, None, None, None, 0.027850467289719738, 6.3041036578, 7.3041036578, 4424]


# set size

In [16]:
def getCompaniesInMonth(month):
  compainesInMonth = []
  for tickerKey in companyInfoDict.keys():
    if month in companyInfoDict[tickerKey]:
      compainesInMonth.append(companyInfoDict[tickerKey][month])
  return compainesInMonth

In [17]:
import math
for month in allMonths:
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.rollingAverage is not None and x.marketcap is not None]
    companiesInMonth.sort(key=operator.attrgetter("marketcap"), reverse=True)
    median = math.ceil(len(companiesInMonth) / 2)
    for company in companiesInMonth[0:median]:
        company.size = "Big"
    for company in companiesInMonth[median:]:
        company.size = "Small"

print ([x for x in getCompaniesInMonth(allMonths[-1]) if x.size ][-10:])

[tickerKey:251	 month:140004	 closeAdjust:37960.0	 currentYield:0.04573002754820932	 numberOfOutstandingShares:840000000	 marketcap:31760400000000	 b2m:None	 rollingAverage:-0.053558532559853156	 momentum:None	 size:Big
, tickerKey:255	 month:140004	 closeAdjust:14660.0	 currentYield:-0.011463250168577188	 numberOfOutstandingShares:1279000000	 marketcap:19121050000000	 b2m:None	 rollingAverage:-0.07897654465431342	 momentum:None	 size:Small
, tickerKey:259	 month:140004	 closeAdjust:35270.0	 currentYield:0.054409566517189756	 numberOfOutstandingShares:360000000	 marketcap:12870000000000	 b2m:None	 rollingAverage:-0.0650800251564716	 momentum:None	 size:Small
, tickerKey:263	 month:140004	 closeAdjust:1884.0	 currentYield:0.08525345622119818	 numberOfOutstandingShares:24131803999	 marketcap:45054078066133	 b2m:None	 rollingAverage:-0.06532536253116837	 momentum:None	 size:Big
, tickerKey:267	 month:140004	 closeAdjust:10020.0	 currentYield:0.08559046587215602	 numberOfOutstandingShares:

In [18]:

    
    
for index,month in enumerate(allMonths[:-1]):
    companiesInMonth = [x for x in getCompaniesInMonth(month) if x.rollingAverage is not None and x.marketcap is not None]
    companiesInMonth = [x for x in companiesInMonth if findCompanyInfo(x.tickerKey,allMonths[index+1]) is not None and findCompanyInfo(x.tickerKey,allMonths[index+1]).currentYield is not None]
    companiesInMonth.sort(key=operator.attrgetter("rollingAverage"), reverse=True)
    count = math.floor(len(companiesInMonth) / 3) #It might be better to round it up instead of down to increase valid data
    for company in companiesInMonth[0:count]:
        company.momentum = "High"
    for company in companiesInMonth[len(companiesInMonth) - count:]:
        company.momentum = "Low"
    for company in companiesInMonth[count:len(companiesInMonth) - count]:
        company.momentum = "Notural"

print ([x for x in getCompaniesInMonth(allMonths[-2]) if x.momentum is not None ][-10:])

[tickerKey:251	 month:140003	 closeAdjust:36300.0	 currentYield:-0.07539480387162512	 numberOfOutstandingShares:840000000	 marketcap:30508800000000	 b2m:9.2280538648	 rollingAverage:-0.08849092161533151	 momentum:Low	 size:Big
, tickerKey:255	 month:140003	 closeAdjust:14830.0	 currentYield:-0.187842278203724	 numberOfOutstandingShares:1279000000	 marketcap:19069890000000	 b2m:7.3866604045	 rollingAverage:-0.029776004360683044	 momentum:High	 size:Small
, tickerKey:259	 month:140003	 closeAdjust:33450.0	 currentYield:-0.28875186051456514	 numberOfOutstandingShares:360000000	 marketcap:12189600000000	 b2m:5.1790563469	 rollingAverage:-0.022108740858892844	 momentum:High	 size:Small
, tickerKey:263	 month:140003	 closeAdjust:1736.0	 currentYield:-0.032329988851728	 numberOfOutstandingShares:24131803999	 marketcap:41651493702274	 b2m:1.251196135	 rollingAverage:-0.10872790139556543	 momentum:Low	 size:Big
, tickerKey:267	 month:140003	 closeAdjust:9230.0	 currentYield:-0.02637130801687759

In [19]:
def avgTriple(first, second, third):
    avg = 0
    count = 0
    if first is not None:
        count += 1
        avg += first
    if second is not None:
        count += 1
        avg += second
    if third is not None:
        count += 1
        avg += third
    if count != 0:
        avg /= count
    return avg

# MomentomFactor factor (MOM)

In [20]:


MoMFactor = {}

for index,month in enumerate(allMonths[:-1]):
    # Get companies of corrent month
    companiesInMonth = getCompaniesInMonth(month)
    # Get companis for these filters SL SH BL BH
    SL = [company for company in companiesInMonth if company.size == "Small" and company.momentum == "Low"]
    SH = [company for company in companiesInMonth if company.size == "Small" and company.momentum == "High"]
    BL = [company for company in companiesInMonth if company.size == "Big" and company.momentum == "Low"]
    BH = [company for company in companiesInMonth if company.size == "Big" and company.momentum == "High"]
    # Get companies next month info
    BH_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BH]
    BL_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in BL]
    SH_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SH]
    SL_nextMonth = [findCompanyInfo(company.tickerKey,allMonths[index+1]) for company in SL]
    # Get companies next month currentYield
    BH_average = average([x for x in BH_nextMonth if x is not None])
    BL_average = average([x for x in BL_nextMonth if x is not None])
    SH_average = average([x for x in SH_nextMonth if x is not None])
    SL_average = average([x for x in SL_nextMonth if x is not None])
    
    MoMFactor[month]=avgTriple(SH_average, BH_average,None) - avgTriple(SL_average, BL_average,None)

print(MoMFactor)


{138008: -0.10811388388764293, 138009: 0.1341950241247694, 138010: -0.013236726183325248, 138011: 0.13910395781762366, 138102: 0.05060203706891442, 138103: 0.03578384817674013, 138108: -0.037996426252533994, 138109: -0.06735904449901281, 138110: 0.026328142891619333, 138111: 0.07483217433129774, 138112: -0.00837665618342931, 138201: 0.08962260192987398, 138202: 0.08037159425114951, 138203: 0.16723490774257527, 138204: 0.28573276373668755, 138209: -0.00907953051077276, 138211: -0.04788706726276215, 138212: -0.05245349483114489, 138301: 0.06270642224238937, 138302: -0.13975740186798882, 138303: 0.05211980248889178, 138304: -0.05021346874857735, 138305: 0.3862988174658408, 138306: 0.053370180179440674, 138307: 0.10142182928143467, 138308: -0.22190496010551997, 138309: -0.06717768023290388, 138310: 0.056944150120439546, 138311: -0.06834891751725974, 138401: -0.016928813805789478, 138402: -0.027835700891215176, 138406: 0.044941841134925414, 138407: -0.03838987467089383, 138408: 0.0213774241

# Book to market Calc

In [21]:
#  Fill all marketcap based on the last one
for companyTicker in companyTickerSet:
  for index,month in enumerate(allMonths):
    if month not in companyInfoDict[companyTicker]:
      firstData:CompanyInfo = companyInfoDict[companyTicker][list(companyInfoDict[companyTicker].keys())[0]]
      c = CompanyInfo(companyTicker,firstData.name,month,None,None,None,None,None,None, firstData.sourceSubsectorCode)
      if index > 0 and allMonths[index-1] in companyInfoDict[companyTicker]:
        c.marketcap = companyInfoDict[companyTicker][allMonths[index-1]].marketcap
        c.b2m = companyInfoDict[companyTicker][allMonths[index-1]].b2m
        c.inv = companyInfoDict[companyTicker][allMonths[index-1]].inv
        c.op = companyInfoDict[companyTicker][allMonths[index-1]].op
      companyInfoDict[companyTicker][month] = c
  # print(companyInfoDict[companyTicker][139712])

for month in sorted(companyInfoDict[1].keys())[:10]:
  print(companyInfoDict[1][month])

tickerKey:1	 month:138001	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 size:None

tickerKey:1	 month:138002	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 size:None

tickerKey:1	 month:138003	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 size:None

tickerKey:1	 month:138004	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 size:None

tickerKey:1	 month:138005	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:None	 size:None

tickerKey:1	 month:138006	 closeAdjust:None	 currentYield:None	 numberOfOutstandingShares:None	 marketcap:None	 b2m:None	 rollingAverage:None	 momentum:Non

In [22]:
for companyTicker in companyTickerSet:
  lastB2MOfTheYear = None
  lastINVOfTheYear = None
  lastOpOfTheYear = None
  for month in allMonths:
    if month % 100 == 5:
      lastB2MOfTheYear = companyInfoDict[companyTicker][month].b2m
      lastINVOfTheYear = companyInfoDict[companyTicker][month].inv
      lastOpOfTheYear = companyInfoDict[companyTicker][month].op
    companyInfoDict[companyTicker][month].b2m = lastB2MOfTheYear     
    companyInfoDict[companyTicker][month].inv = lastINVOfTheYear     
    companyInfoDict[companyTicker][month].op = lastOpOfTheYear     



In [23]:
# for month in sorted(companyInfoDict[1].keys())[:10]:
#   print(companyInfoDict[9][month])

In [24]:
# Book To market ranking

for month in allMonths:
  b2mMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].b2m == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    b2mMonthList.append(companyInfoDict[companyTicker][month].b2m)
  if len(b2mMonthList) == 0 :
    continue
  b2mMonthList.sort()
  # print(len(b2mMonthList),int(len(b2mMonthList)*0.3))
  lowBookToMarket = b2mMonthList[int(len(b2mMonthList)*0.3)]
  highBookToMarket = b2mMonthList[int(-len(b2mMonthList)*0.3)]

  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].b2m == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    if companyInfoDict[companyTicker][month].b2m <= lowBookToMarket:
      companyInfoDict[companyTicker][month].BperMSize = "Low"
    elif companyInfoDict[companyTicker][month].b2m >= highBookToMarket:
      companyInfoDict[companyTicker][month].BperMSize = "High"
    else:
      companyInfoDict[companyTicker][month].BperMSize = "Neutral"



In [25]:
# RMW (Operational profit) ranking

for month in allMonths:
  dataMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].op == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    dataMonthList.append(companyInfoDict[companyTicker][month].op)
  if len(dataMonthList) == 0 :
    continue
  dataMonthList.sort()
  # print(len(dataMonthList),int(len(dataMonthList)*0.3))
  weakValue = dataMonthList[int(len(dataMonthList)*0.3)]
  robustValue = dataMonthList[int(-len(dataMonthList)*0.3)]
  # if month == 139902:
  #   print(dataMonthList)
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].op == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    if companyInfoDict[companyTicker][month].op <= weakValue:
      companyInfoDict[companyTicker][month].RMWSize = "Weak"
    elif companyInfoDict[companyTicker][month].op >= robustValue:
      companyInfoDict[companyTicker][month].RMWSize = "Robust"
    else:
      companyInfoDict[companyTicker][month].RMWSize = "Neutral"

In [26]:

for companyTicker in companyTickerSet[:10]:
  print(companyInfoDict[companyTicker][139807].RMWSize)

Neutral
Neutral
Weak
Neutral
Robust
Weak
Weak
Robust
Robust
Neutral


In [27]:
# CMA (Investment) ranking

for month in allMonths:
  dataMonthList = []
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].inv == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue    
    dataMonthList.append(companyInfoDict[companyTicker][month].inv)
  if len(dataMonthList) == 0 :
    continue
  dataMonthList.sort()
  # print(len(dataMonthList),int(len(dataMonthList)*0.3))
  weakValue = dataMonthList[int(len(dataMonthList)*0.3)]
  robustValue = dataMonthList[int(-len(dataMonthList)*0.3)]
  # if month == 139902:
  #   print(dataMonthList)
  for companyTicker in companyTickerSet:
    if companyInfoDict[companyTicker][month].inv == None:
      continue
    if companyInfoDict[companyTicker][month].currentYield == None:
      continue
    if companyInfoDict[companyTicker][month].inv <= weakValue:
      companyInfoDict[companyTicker][month].CMASize = "Conservative"
    elif companyInfoDict[companyTicker][month].inv >= robustValue:
      companyInfoDict[companyTicker][month].CMASize = "Aggressive"
    else:
      companyInfoDict[companyTicker][month].CMASize = "Neutral"

In [28]:
def getNextMonth(currentMonth):
  return allMonths[allMonths.index(currentMonth)+1]

def getNextMonthCompaniesHavingYeild(companyInfoList):
  response = []
  for companyInfo in companyInfoList:
    nextMoonth = companyInfoDict[companyInfo.tickerKey][getNextMonth(companyInfo.month)]
    if nextMoonth.currentYield != None:
      response.append(nextMoonth)
  return response


In [29]:
HMLFactor = {}
SMBBTMFactor = {}
for month in allMonths[:-1]:
  SH = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "High" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Neutral" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  SL = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Low" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  BH = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "High" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Neutral" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  BL = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].BperMSize == "Low" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  
  
  SH_average = average(getNextMonthCompaniesHavingYeild(SH))
  SN_average = average(getNextMonthCompaniesHavingYeild(SN))
  SL_average = average(getNextMonthCompaniesHavingYeild(SL))
  BH_average = average(getNextMonthCompaniesHavingYeild(BH))
  BN_average = average(getNextMonthCompaniesHavingYeild(BN))
  BL_average = average(getNextMonthCompaniesHavingYeild(BL))
  
  SMBBTMFactor[month] = avgTriple(SH_average,SN_average,SL_average) - avgTriple(BH_average, BN_average, BL_average)
  HMLFactor[month] = avgTriple(SH_average, BH_average,None)  - avgTriple(SL_average , BL_average,None)
for month in list(HMLFactor.keys())[:10]:
  print(HMLFactor[month])

0
0
0
0
0
0
0.004205131873668888
0.08140214208484109
0.01324088148787228
-0.07933831973756245


In [30]:
RMWFactor = {}
SMBRMWFactor = {}
for month in allMonths[:-1]:
  SR = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Robust" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Neutral" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  SW = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Weak" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  BR = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Robust" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Neutral" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  BW = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].RMWSize == "Weak" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  
  SR_average = average(getNextMonthCompaniesHavingYeild(SR))
  SN_average = average(getNextMonthCompaniesHavingYeild(SN))
  SW_average = average(getNextMonthCompaniesHavingYeild(SW))
  BR_average = average(getNextMonthCompaniesHavingYeild(BR))
  BN_average = average(getNextMonthCompaniesHavingYeild(BN))
  BW_average = average(getNextMonthCompaniesHavingYeild(BW))

  SMBRMWFactor[month] = avgTriple(SR_average , SN_average, SW_average) - avgTriple(BR_average, BN_average, BW_average)
  RMWFactor[month] = avgTriple(SR_average, BW_average,None) - avgTriple(SW_average, BW_average,None)
for month in list(RMWFactor.keys())[:10]:
  print(RMWFactor[month])

0
0
0
0
0
0
-0.04682343234323416
-0.009608016504568329
0.024896265560165998
0.053518044899119044


In [31]:
CMAFactor = {}
SMBCMAFactor = {}
for month in allMonths[:-1]:
  SA = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Aggressive" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  SN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Neutral" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  SC = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Conservative" and \
        companyInfoDict[companyTicker][month].size == "Small"]
  BA = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Aggressive" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  BN = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Neutral" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  BC = [companyInfoDict[companyTicker][month] for companyTicker in companyTickerSet if companyInfoDict[companyTicker][month].CMASize == "Conservative" and \
        companyInfoDict[companyTicker][month].size == "Big"]
  
  SA_average = average(getNextMonthCompaniesHavingYeild(SA))
  SN_average = average(getNextMonthCompaniesHavingYeild(SN))
  SC_average = average(getNextMonthCompaniesHavingYeild(SC))
  BA_average = average(getNextMonthCompaniesHavingYeild(BA))
  BN_average = average(getNextMonthCompaniesHavingYeild(BN))
  BC_average = average(getNextMonthCompaniesHavingYeild(BC))

  SMBCMAFactor[month] = avgTriple(SA_average, SN_average, SC_average) - avgTriple(BA_average, BN_average, BC_average)
  CMAFactor[month] = avgTriple(SC_average, BC_average,None)  - avgTriple(SA_average, BA_average,None)
for month in list(CMAFactor.keys())[:10]:
  print(CMAFactor[month])

0
0
0
0
0
0
-0.004205131873668888
-0.08140214208484109
-0.01324088148787228
0.07933831973756245


In [32]:
SMBFactor = {}
for month in SMBBTMFactor.keys():
  SMBFactor[month] = (SMBBTMFactor[month] + SMBCMAFactor[month] + SMBRMWFactor[month]) / 3


# Export company list to excel

In [33]:
# Example to excel function
exportCompanyListToExcel("calculations")

In [34]:
# Factors to excel
rows = []
for month in allMonths:
     rows.append([
       month,
       SMBFactor[month] if month in SMBFactor else None,
       HMLFactor[month] if month in HMLFactor else None,
       RMWFactor[month] if month in RMWFactor else None,
       CMAFactor[month] if month in CMAFactor else None,
       MoMFactor[month] if month in MoMFactor else None,
     ])
dfOut = pd.DataFrame(rows, columns=['month',"SMBFactor", "HMLFactor", "RMWFactor", "CMAFactor", "MoMFactor"])
dfOut.to_excel('Factors.xlsx', sheet_name='Factors')